In [1]:
import datetime
import os
import boto3
from botocore import UNSIGNED
from botocore.config import Config

In [3]:
def download_CMI(yyyymmddhhmn, band, path_dest, product_name):

  os.makedirs(path_dest, exist_ok=True)

  year = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
  day_of_year = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%j')
  hour = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
  min = datetime.datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

  # AMAZON repository information
  # https://noaa-goes16.s3.amazonaws.com/index.html
  bucket_name = 'noaa-goes16'
  #product_name = 'ABI-L2-CMIPF'

  # Initializes the S3 client
  s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
  #-----------------------------------------------------------------------------------------------------------
  # File structure
  prefix = f'{product_name}/{year}/{day_of_year}/{hour}/OR_{product_name}-M6C{int(band):02.0f}_G16_s{year}{day_of_year}{hour}{min}'

  # Seach for the file on the server
  s3_result = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix, Delimiter = "/")

  #-----------------------------------------------------------------------------------------------------------
  # Check if there are files available
  if 'Contents' not in s3_result:
    # There are no files
    print(f'No files found for the date: {yyyymmddhhmn}, Band-{band}')
    return -1
  else:
    # There are files
    for obj in s3_result['Contents']:
      key = obj['Key']
      # Print the file name
      file_name = key.split('/')[-1].split('.')[0]

      # Download the file
      if os.path.exists(f'{path_dest}/{file_name}.nc'):
        print(f'File {path_dest}/{file_name}.nc exists')
      else:
        print(f'Downloading file {path_dest}/{file_name}.nc')
        s3_client.download_file(bucket_name, key, f'{path_dest}/{file_name}.nc')
  return f'{file_name}'

In [10]:
# single file download
path_dest = 'F:/GOES_Files/'
date_save = '20240904'
time_save = '2120'
yyyymmddhhmn = date_save + time_save
Band = 5
file1 = download_CMI(yyyymmddhhmn, Band, path_dest, 'ABI-L2-CMIPF')
path = 'content/GOES_Files/' + file1 + '.nc'

In [ ]:
# download complete days between two dates
path_dest = 'F:/GOES_Files/'
path_dest = 'D:/RODRIGO/IntradayForecasting/content/GOES_Files/ABI_L2_CMIPF'


start_date = '20230517'  # Starting date in 'YYYYMMDD' format
end_date = '20230517'    # Ending date in 'YYYYMMDD' format
Band = 13

start_date = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date = datetime.datetime.strptime(end_date, '%Y%m%d')

current_date = start_date
while current_date <= end_date:
    date_save = current_date.strftime('%Y%m%d') 
    for hour in range(0, 24):
        for minute in range(0, 60, 10):
            time_save = f'{hour:02d}{minute:02d}'
            yyyymmddhhmn = date_save + time_save
            file1 = download_CMI(yyyymmddhhmn, Band, path_dest, 'ABI-L2-CMIPF')
    current_date += datetime.timedelta(days=1)

In [ ]:
# Download hourly data
path_dest = 'F:/GOES_Files/GOES_Files_B02'
# path_dest = 'D:/RODRIGO/IntradayForecasting/content/GOES_Files/ABI_L2_CMIPF'

start_date = '20230501'  # Starting date in 'YYYYMMDD' format
end_date = '20230531'    # Ending date in 'YYYYMMDD' format
Band = 2

start_date = datetime.datetime.strptime(start_date, '%Y%m%d')
end_date = datetime.datetime.strptime(end_date, '%Y%m%d')

current_date = start_date
while current_date <= end_date:
    date_save = current_date.strftime('%Y%m%d') 
    for hour in range(0, 24):
        for minute in range(0, 60, 60):  # Change from range(0, 60, 10) to range(0, 60, 60) to iterate every hour
            time_save = f'{hour:02d}{minute:02d}'
            yyyymmddhhmn = date_save + time_save
            file1 = download_CMI(yyyymmddhhmn, Band, path_dest, 'ABI-L2-CMIPF')
    current_date += datetime.timedelta(days=1)